In [2]:
%matplotlib inline

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import albumentations as A
import cv2
import numpy as np
import os

import matplotlib.pyplot as plt

In [4]:
# Define the directory containing your images
image_dir = "/content/drive/MyDrive/Projects/Cricket_bat_Corner_Detection/Images"

In [5]:
# List image file paths in the directory
image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith((".jpg", ".jpeg", ".png"))]

In [6]:
counter = 110

In [7]:
# Define the directory containing your labels
txt_dir = "/content/drive/MyDrive/Projects/Cricket_bat_Corner_Detection/Labels"

# List label file paths in the directory
file_names = [os.path.join(txt_dir, filename) for filename in os.listdir(txt_dir) if filename.endswith((".txt"))]

In [8]:
# Define a function for converting the bounding boxes output from augmentation

def single_obj_bb_yolo_conversion(transformed_bboxes, class_names):
    if len(transformed_bboxes):
        class_num = class_names.index(transformed_bboxes[-1])
        bboxes = list(transformed_bboxes)[:-1] # .insert(0, '0')
        bboxes.insert(0, class_num)
    else:
        bboxes = []
    return bboxes


def multi_obj_bb_yolo_conversion(aug_labs, class_names):
    yolo_labels = []
    for aug_lab in aug_labs:
        bbox = single_obj_bb_yolo_conversion(aug_lab, class_names)
        yolo_labels.append(bbox)
    return yolo_labels

In [9]:
# Define a function for saving the converted boiunding box values in yolo format

def save_aug_lab(transformed_bboxes, lab_pth, lab_name):
    lab_out_pth = os.path.join(lab_pth, lab_name)
    with open(lab_out_pth, 'w') as output:
        for bbox in transformed_bboxes:
            updated_bbox = str(bbox).replace(',', ' ').replace('[', '').replace(']', '')
            output.write(updated_bbox + '\n')


In [11]:
# Define the directory for saving new label files
path = '/content/drive/MyDrive/Projects/Cricket_bat_Corner_Detection/Labels_New'
class_names = ["c"]

In [12]:
# Define a function to process each line and create arrays
def process_line(line):
    # Split the line by spaces and convert the values to floats
    values = [float(val) for val in line.strip().split() if val != '0']

    sub_array = values + ['c']  # Add 'c' at the end of the sub-array

    return sub_array

In [19]:
for i in range(len(image_paths)):

    # Read the image using OpenCV
    image = cv2.imread(image_paths[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


    with open(file_names[i], "r") as file:
        result = []
        # Read the file line by line
        for line in file:
            if line.strip():  # Check if the line is not empty
                sub_array = process_line(line)
                result.append(sub_array)

    # Check if the image was successfully loaded
    if image is not None:

        transform = A.Compose([A.OneOf([
                                A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),
                                A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1),
                                ])], p=0.5,bbox_params=A.BboxParams(format='yolo'))

        transformed = transform(image=image, bboxes=result)

        new_filename = f"{counter:04d}"

        cv2.imwrite(f"/content/drive/MyDrive/Projects/Cricket_bat_Corner_Detection/Images_New/{new_filename}.jpg", cv2.cvtColor(transformed['image'], cv2.COLOR_BGR2RGB))

        convert = multi_obj_bb_yolo_conversion(transformed['bboxes'], class_names)
        label_name = f"{counter:04d}.txt"
        save_aug_lab(convert,path,label_name)

    else:
        print(f"Error reading image: {image_paths}")

    counter += 1

## Augmentations that was used in images

In [ ]:
#1. transform = A.Compose([
        # A.HorizontalFlip(p=0.5),
        # A.ShiftScaleRotate(p=0.5),
        # A.RandomBrightnessContrast(p=0.3),
        # A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.3),],bbox_params=A.BboxParams(format='yolo'),)

#2. transform = A.Compose([
                      # A.MotionBlur(p=3),
                      # A.MedianBlur(blur_limit=5, p=0.3),
                      # A.Blur(blur_limit=8, p=0.7)], bbox_params=A.BboxParams(format='yolo'))

#3. transform = A.Compose([
                      # A.RandomScale(scale_limit=0.2, p=0.5),
                      # A.RandomRotate90(p=0.5),
                      # A.HorizontalFlip(p=0.5)], bbox_params=A.BboxParams(format='yolo'))

#4. transform =  A.Compose([ A.RandomBrightnessContrast(p=0.5),
                                  # A.RandomGamma(p=0.5),
                                  # A.CLAHE(p=0.3)],bbox_params=A.BboxParams(format='yolo'))

#5. transform =  A.Compose([A.RandomResizedCrop(height=512, width=512, scale=(0.8, 1.0), p=0.3)],bbox_params=A.BboxParams(format='yolo'))

#6. transform = A.Compose([
                # A.RandomScale(scale_limit=0.2, p=0.5),
                # A.HorizontalFlip(p=0.5),
                # A.RandomBrightnessContrast(p=0.5),], p=0.5,bbox_params=A.BboxParams(format='yolo'))

#7.  transform = A.Compose([A.OneOf([
                                # A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),
                                # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1),
                                # ])], p=0.5,bbox_params=A.BboxParams(format='yolo'))